# Solar Panels Detection

Training and Detection of solar panels using YOLOv8 and the MAXAR WorldView-3 30 cm dataset (Germany region).

The original dataset was split into train, validation, and test. This step can be done with the notebook *solar_dataset_preparation.ipynb*

## Setup

In [1]:
!pip -q install ultralytics # Ignore if not using Colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## Import Libraries

In [14]:
from ultralytics import YOLO
import os
import torch
import yaml
try:
    from google.colab import files
    from google.colab import drive
except:
    print("Some libraries were ignored.")

## User Inputs

In [12]:
# Datasets folder
# Folder where the train, validation and test datasets are saved
datasets_folder = "datasets"

# YOLO model hyperparameters
# Target image size for training
image_size = 416
# Total number of training epochs
epochs_num = 100
# Batch size for training. -1 is auto.
batch_size = -1
# Minimum confidence threshold for detections
conf_thr = 0.2
# Intersection Over Union threshold
iou_thr = 0.7

## Code

### Train and Validation

Ignore the **Train and Validation** step if you already trained the model.

In [4]:
# Create new folder (if does not exist) to store the datasets
if not os.path.exists(datasets_folder):
  os.mkdir(datasets_folder)

In [5]:
# Copies the Train, Validation and Test saved on datasets folder from your Drive to this VM
try:
  drive.mount("/content/drive")
  save_driver_folder = "/content/drive/MyDrive/datasets"
  # Train
  !cp {save_driver_folder+"/train.zip"} {datasets_folder}

  # Validation
  !cp {save_driver_folder+"/validation.zip"} {datasets_folder}

  # Test
  !cp {save_driver_folder+"/test.zip"} {datasets_folder}
except:
  print("Copy of datasets from Drive ignored.")

Mounted at /content/drive


In [6]:
# Unzip the datasets
# Train
!unzip -q -o {datasets_folder+"/train.zip"} -d {datasets_folder}
os.remove(datasets_folder+"/train.zip")

# Validation
!unzip -q -o {datasets_folder+"/validation.zip"} -d {datasets_folder}
os.remove(datasets_folder+"/validation.zip")

# Test
!unzip -q -o {datasets_folder+"/test.zip"} -d {datasets_folder}
os.remove(datasets_folder+"/test.zip")

In [8]:
# Check if GPU is available
if torch.cuda.is_available():
  print("GPU available.")
  torch.cuda.set_device(0)
  device = 0 # Single GPU, it has support for multiple
else:
  print("GPU not available, using CPU.")
  print("On Colab, go to: Edit > Notebook Settings > Hardware Accelerator > Select T4 GPU")
  torch.cuda.set_device(-1)
  device = "cpu"

GPU available.


In [9]:
# Load best pre-trained YOLOv8 model
yv8x_model = YOLO("models/yolov8x.pt")

100%|██████████| 131M/131M [00:00<00:00, 286MB/s]


In [18]:
# Generate yaml file
yaml_path = os.path.join(datasets_folder, "solar_panels_data.yaml")
yaml_data = {"names":["Solar panels"], "nc":1, "path":os.path.abspath("datasets"),
              "train":"train/images", "val":"validation/images", "test":"test/images"}
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False)

In [19]:
train_results = yv8x_model.train(data=yaml_path, epochs=epochs_num, batch=batch_size,
                                 seed=42, imgsz=image_size, verbose=True, val=True, plots=True, device=device)

Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=models/yolov8x.pt, data=datasets/solar_panels_data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

100%|██████████| 755k/755k [00:00<00:00, 23.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 76.7MB/s]


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=416
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.57G reserved, 0.55G allocated, 13.62G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    68153571       109.1         1.080         57.19         163.3        (1, 3, 416, 416)                    list
    68153571       218.1         1.388          41.6         66.06        (2, 3, 416, 416)                    list
    68153571       436.2         2.024         44.22         71.91        (4, 3, 416, 416)                    list
    68153571       872.5         3.349         82.59         102.1        (8, 3, 416, 416)                    list
    68153571        1745         5.859         153.5         186.7       (16, 3, 416, 416)                    list
AutoBatch: Using batch-size 23 for CUDA:0 9.23G/14.75G (63%) ✅


...

     18/100      10.1G      1.118     0.6405      1.005         89        416: 100%|██████████| 56/56 [00:43<00:00,  1.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:09<00:00,  1.49it/s]

                   all        635       7658      0.922      0.856      0.936      0.681



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     19/100      10.1G      1.067     0.6099     0.9942        433        416:  38%|███▊      | 21/56 [00:16<00:28,  1.24it/s]


KeyboardInterrupt: 

In [18]:
# Copy best weights to Drive
try:
  !cp {"runs/detect/train/weights/best.pt"} {save_driver_folder}
except:
  print("Saving to drive ignored.")

In [ ]:
# Download the best weights
try:
  files.download("runs/detect/train/weights/best.pt")
except:
  print("Download to local machine ignored.")

### Test

In [ ]:
# Import best trained YOLO solar panels model
best_model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
# Try the new model on the Test dataset
test_results = best_model.predict(source=yaml_path, imgsz=image_size, conf=conf_thr,
                                  iou=iou_thr, device=device, show_labels=False,
                                  show_conf=False, save=True)